In [ ]:
import os, random
import numpy as np
import matplotlib.image as pltimage
import matplotlib.pyplot as plt
from PIL import Image
def show_files(label_dir):
    all_files = []
    file_list = os.listdir(label_dir)
    for file in file_list:
        cur_path = os.path.join(label_dir, file)
        root, extension = os.path.splitext(cur_path)
        ext_lower = extension.lower()
        if ext_lower == ".png" or ext_lower == ".jpg" or ext_lower == ".jpeg":
            all_files.append(cur_path)
    return all_files

def obtainXY(data_dir, shuffle=False):
    X = []
    Y = []
    classes =  [i for i in range(10)]
    for label in classes:
        all_files = show_files(data_dir + str(label))
        for file in all_files:
            img = Image.open(file)
            img = img.resize((28, 28), Image.ANTIALIAS)
            gray = img.convert("L")
            x = np.asarray(gray)/255.
            y = label
            X.append(x)
            Y.append(y)
    X = np.array(X)
    Y = np.array(Y)
    if shuffle:
        indexs = [i for i in range(len(X))] 
        random.shuffle(indexs) #乱序
        X = X[indexs]
        Y = Y[indexs]
    return X, Y

dataset_dir = "./dataset/"
X, Y = obtainXY(dataset_dir, shuffle=True)
print(X.shape, Y.shape)

In [ ]:
from collections import  Counter

def print_pecent(data, class_num=10):
    y_ = list(data)
    size = len(y_)
    class_counter = Counter(y_)
    print(class_counter, size)
    for i in range(class_num):
        print(i, "占比:", round(class_counter[i]/size, 3))

print_pecent(Y)

In [ ]:
import tensorflow as tf
from keras import layers, regularizers
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

def creat_nn(print_summary=False):
    model = Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', 
        optimizer='adam', metrics=['accuracy'])
    if print_summary:
        model.summary()
    return model

def train_nn(train_x, train_y, batch_size, epochs, load_save_model=False):
    model_path = './model/'
    if load_save_model:
        print('train load save model path', model_path)
        model = load_model(model_path)
    else:
        print('train create new model')
        model = creat_nn(print_summary=True)
    model.fit(train_x, train_y, batch_size=batch_size, 
        epochs=epochs, verbose=1)
    model.save(model_path)
    
    #save tflite model
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    open("mymodel.tflite", "wb").write(tflite_model)
    return model

train_x = X
# 将label转成向量，如 2 => [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
train_y = tf.keras.utils.to_categorical(Y, num_classes=10)
# input output shape
print(train_x.shape, train_y.shape)
# first train model
model = train_nn(train_x, train_y, batch_size=16, epochs=16, load_save_model=False)

In [ ]:
def verify_model(model, valid_x, valid_y, num_classes=10):
    y_pred = model.predict(valid_x)
    y_pred = np.argmax(y_pred, axis=1)
    size = [0 for i in range(num_classes)]
    correct = [0 for i in range(num_classes)]
    for i in range(len(y_pred)):
        label = valid_y[i]
        pred = y_pred[i]
        size[label] += 1
        if pred == label:
            correct[label] += 1
    print("total accuracy:", sum(correct)/sum(size))
    for i in range(num_classes):
        print("label:", str(i), " accuracy:", correct[i]/max(size[i], 1))


dataset_test_dir = "./dataset-test/"
vX, vY = obtainXY(dataset_test_dir)
print(vX.shape, vY.shape)
verify_model(model, vX, vY)

In [ ]:
# retrain model
train_nn(train_x, train_y, batch_size=16, epochs=8, load_save_model=True)

In [ ]:
image = tf.io.read_file("./dataset/0/2022_06_28_08_27_39_796.png")
image = tf.io.decode_jpeg(image)
image = tf.cast(image, tf.float32)
flipped = tf.image.flip_left_right(image)
plt.show(flipped / 255.0)

In [ ]:
def creat_cnn(input_shape=(9, 6)):
    model = Sequential()
    model.add(layers.Conv1D(6, kernel_size=3, input_shape=input_shape, padding='same', activation='relu'))
    model.add(layers.Conv1D(12, kernel_size=3, input_shape=input_shape, padding='same', activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=3, strides=None, padding='same', data_format='channels_last'))
    model.add(layers.Conv1D(24, kernel_size=3, input_shape=input_shape, padding='same', activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    model.summary()
    return model


In [2]:
ls

dataset/                   digital_recognition.ipynb
dataset-test/              model/
dataset-test.zip           model.zip
dataset.zip                mymodel.tflite


In [3]:
import zipfile

def zipDir(zip_file, src_dir, parent_dir):
    files = os.listdir(src_dir)
    for name in files:
        if name not in ['.', '..']:
            cur_name = os.path.join(src_dir, name)
            dst_name = os.path.join(parent_dir, name)
            if os.path.isdir(cur_name):
                zipDir(zip_file, cur_name, os.path.join(parent_dir, dst_name))
            else:
                zip_file.write(cur_name, dst_name)

zip_file = 'dataset.zip'
z = zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED)
zip_dir_name = os.path.join("./", "dataset")
zipDir(z, zip_dir_name, "")
z.close()

TypeError: expected str, bytes or os.PathLike object, not function